In [6]:
# !mkdir hyperopt config
# !mkdir -pv hyperopt/config  hyperopt/hyperopt
# conda activate pkmodel
# /home/atest/anaconda3/envs/pkmodel/bin/python train_caco2_rdkit_search.py
# /home/atest/anaconda3/envs/pkmodel/bin/python /home/zonghu.wang/workspace/pbpk/chemprop/hyperopt/train_cl_morgan_search.py
# train_cl_morgan_search.py
/home/atest/anaconda3/envs/pkmodel/bin/python train_cl_morgan_search.py
/home/atest/anaconda3/envs/pkmodel/bin/python train_caco2_search.py
/home/atest/anaconda3/envs/pkmodel/bin/python train_caco2_rdkit_search.py
# /home/atest/anaconda3/envs/pkmodel/bin/python /home/zonghu.wang/workspace/pbpk/chemprop/hyperopt/train_caco2_morgan_search.py
# /home/atest/anaconda3/envs/pkmodel/bin/python -m pip install git+https://github.com/bp-kelley/descriptastorus
# #     /data/anaconda3/envs/pkmodel/bin/chemprop_fingerprint
# #     /data/anaconda3/envs/pkmodel/bin/chemprop_hyperopt
# #     /data/anaconda3/envs/pkmodel/bin/chemprop_interpret
# #     /data/anaconda3/envs/pkmodel/bin/chemprop_predict
# #     /data/anaconda3/envs/pkmodel/bin/chemprop_train
# #     /data/anaconda3/envs/pkmodel/bin/chemprop_web
# #     /data/anaconda3/envs/pkmodel/bin/sklearn_predict
# #     /data/anaconda3/envs/pkmodel/bin/sklearn_train
# #     /data/anaconda3/envs/pkmodel/lib/python3.10/site-packages/chemprop-1.5.2.dist-info/*
# #     /data/anaconda3/envs/pkmodel/lib/python3.10/site-packages/chemprop/*
# mv  /data/anaconda3/envs/pkmodel/lib/python3.10/site-packages/chemprop-1.5.2.dist-info /data/anaconda3/envs/pkmodel/lib/python3.10/site-packages/chemprop-1.5.2.dist-info-bak

mkdir: cannot create directory ‘hyperopt’: File exists
mkdir: cannot create directory ‘config’: File exists


# train_fu_search

In [2]:
%%writefile hyperopt/train_fu_search.py
import numpy as np
import pandas as pd
from parameterized import parameterized
from io import BytesIO
import json
import os
# from chemprop.utils import StopWatch
from tempfile import TemporaryDirectory
from typing import List
import chemprop
from chemprop.constants import TEST_SCORES_FILE_NAME
from chemprop.hyperparameter_optimization import chemprop_hyperopt,hyperopt
from chemprop.interpret import chemprop_interpret
from chemprop.sklearn_predict import sklearn_predict
from chemprop.sklearn_train import sklearn_train
from chemprop.train import chemprop_train, chemprop_predict, evaluate_predictions, chemprop_fingerprint
from chemprop.web.wsgi import build_app
from chemprop.spectra_utils import normalize_spectra, load_phase_mask
from chemprop.features import load_features

from io import BytesIO
import json
import os
from tempfile import TemporaryDirectory
from typing import List
import unittest
from unittest import TestCase
from unittest.mock import patch
 
from loguru import logger
SEED = 2023
EPOCHS = 10
NUM_FOLDS = 5
NUM_ITER = 2
SIZE = 10
DEPTH = 3
DELTA = 0.025
train_param=""
def run_hyperopt(data_path,data_name,target_column,smiles_column,batch_size=64,epochs=100):
    flags: List[str] = None
    with TemporaryDirectory() as save_dir:
                # Train
                config_save_path = os.path.join(save_dir, 'config.json')
                dataset_type="regression"
             
                # Check results

                # parameters = {'epochs':{50,100,200},'batch_size':{16,32,64,128,256,512,1024},'depth': (2, 6), 'hidden_size': (300, 2400), 'ffn_hidden_size': (300, 2400), 'ffn_num_layers': (1, 3), 'dropout': (0.0, 0.4)}
            
                # parameters = {'depth': (2, 6), 'hidden_size': (300, 2400), 'ffn_hidden_size': (300, 2400), 'ffn_num_layers': (1, 3), 'dropout': (0.0, 0.4)}
                # parameters = {'epochs':{50,100,200},'batch_size':{16,32,64,128,256,512,1024},'depth': (2, 6), 'hidden_size': (300, 1200),
                #               'ffn_hidden_size': (300, 1200), 'ffn_num_layers': (1, 3), 'dropout': (0.0,0.2, 0.4)}
                # parameters = {'epochs':{50,100,200},'batch_size':{16,32,64,128,256,512,1024},'depth': (2, 5), 'dropout': (0.0, 0.2, 0.4)}
                # parameters = {'batch_size':{16,32,64,128,256,512,1024},'depth': (2, 5), 'dropout': (0.0, 0.2, 0.4)}
                # parameters = {'batch_size':{16,32,64,128,256,512,1024},'depth': (2, 5), 'dropout': (0.0, 0.2, 0.4)}
                # logger.info("parameters:%s"%parameters)
            
            
                raw_args = [
                  '--data_path',data_path,
                  '--dataset_type', dataset_type,
                  '--save_dir', "hyperopt/"+data_name,
                  '--target_columns', target_column,
                  '--smiles_column',smiles_column,
                  '--seed', str(SEED),
                  '--loss_function','mse',
                 # '--checkpoint_dir','ck_dir',
                  '--split_type', 'random', # Literal['random', 'scaffold_balanced', 'predetermined', 'crossval', 'cv', 'cv-no-test', 'index_predetermined', 'random_with_repeated_smiles'] = 'random'
                  '--split_sizes', '0.8', '0.1', '0.1',
                  '--num_folds', str(NUM_FOLDS),
                  '--epochs', '%s'%epochs,
                  '--metric', 'rmse',
                  '--empty_cache',
                  '--extra_metrics', 'r2', 'mae',
                  '--batch_size','%s'%batch_size,
                  '--save_smiles_splits',
                  '--config_save_path',"config/"+data_name,
                  '--num_iter','100',
                  '--empty_cache',
 '--search_parameter_keywords','basic',
                  '--quiet'] 
    # Hyperopt
    # with patch('sys.argv', raw_args):
                # command_line = ' '.join(raw_args[:])
                # logger.info(f'python hyperparameter_optimization.py {command_line}')

                args = chemprop.args.HyperoptArgs().parse_args(raw_args)
                logger.info("args:%s"%args)
                hyperopt(args=args)

                # for parameter, (min_value, max_value) in parameters.items():
                #     self.assertTrue(min_value <= config[parameter] <= max_value)
    # args = chemprop.args.TrainArgs().parse_args(arguments)
    # mean_score, std_score = chemprop.train.cross_validate(args=args, train_func=chemprop.train.run_training)
    logger.info(raw_args)
    logger.info("data_name:%s data_path:%s target_column:%s"%(data_name,data_path,target_column))
    # logger.info("mean_score:%s std_score:%s"%(mean_score,std_score))

dataset_dir= '/data/project/pbpk/data/'
dataset_name='fup_data_human_2668.csv'
data_path=dataset_dir+dataset_name
logger.info(dataset_dir+dataset_name)
smiles_column='SMILES'
target_column= 'fup_log10'
data_name='fup_log10_opt'
# train(dataset_file,data_name,target_column,smiles_column,batch_size=64,epochs=50)
# stopwatch = StopWatch()
run_hyperopt(data_path,data_name,target_column,smiles_column,batch_size=32,epochs=100)
# stopwatch.stop('hyper search finish: ')

# python hyperparameter_optimization.py /data/project/pbpk/data/fup_data_human_2668.csv --dataset_type regression --save_dir result/fup_log10 
# --target_columns fup_log10 --smiles_column SMILES --loss_function mse --split_type random --split_sizes 0.8 0.1 0.1 --num_folds 5 --epochs 20 
# --batch_size 32 --save_smiles_splits --config_save_path config/fup_log10 --num_iter 10 --empty_cache --quiet


Overwriting hyperopt/train_fu_search.py


## train_fu_rdkit_search

In [3]:
%%writefile hyperopt/train_fu_rdkit_search.py
import sys
sys.path.append("/data/project/pbpk/chemprop")
import numpy as np
import pandas as pd
from parameterized import parameterized
from io import BytesIO
import json
import os
from chemprop.utils import StopWatch
from tempfile import TemporaryDirectory
from typing import List
import chemprop
from chemprop.constants import TEST_SCORES_FILE_NAME
from chemprop.hyperparameter_optimization import chemprop_hyperopt,hyperopt
from chemprop.interpret import chemprop_interpret
from chemprop.sklearn_predict import sklearn_predict
from chemprop.sklearn_train import sklearn_train
from chemprop.train import chemprop_train, chemprop_predict, evaluate_predictions, chemprop_fingerprint
from chemprop.web.wsgi import build_app
from chemprop.spectra_utils import normalize_spectra, load_phase_mask
from chemprop.features import load_features

from io import BytesIO
import json
import os
from tempfile import TemporaryDirectory
from typing import List
import unittest
from unittest import TestCase
from unittest.mock import patch
 
from loguru import logger
SEED = 2023
EPOCHS = 10
NUM_FOLDS = 5
NUM_ITER = 2
SIZE = 10
DEPTH = 3
DELTA = 0.025
train_param=""
def run_hyperopt(data_path,data_name,target_column,smiles_column,batch_size=64,epochs=100):
    flags: List[str] = None
    with TemporaryDirectory() as save_dir:
                # Train
                config_save_path = os.path.join(save_dir, 'config.json')
                dataset_type="regression"
             
                # Check results

                # parameters = {'epochs':{50,100,200},'batch_size':{16,32,64,128,256,512,1024},'depth': (2, 6), 'hidden_size': (300, 2400), 'ffn_hidden_size': (300, 2400), 'ffn_num_layers': (1, 3), 'dropout': (0.0, 0.4)}
            
                # parameters = {'depth': (2, 6), 'hidden_size': (300, 2400), 'ffn_hidden_size': (300, 2400), 'ffn_num_layers': (1, 3), 'dropout': (0.0, 0.4)}
                # parameters = {'epochs':{50,100,200},'batch_size':{16,32,64,128,256,512,1024},'depth': (2, 6), 'hidden_size': (300, 1200),
                #               'ffn_hidden_size': (300, 1200), 'ffn_num_layers': (1, 3), 'dropout': (0.0,0.2, 0.4)}
                # parameters = {'epochs':{50,100,200},'batch_size':{16,32,64,128,256,512,1024},'depth': (2, 5), 'dropout': (0.0, 0.2, 0.4)}
                # parameters = {'batch_size':{16,32,64,128,256,512,1024},'depth': (2, 5), 'dropout': (0.0, 0.2, 0.4)}
                # parameters = {'batch_size':{16,32,64,128,256,512,1024},'depth': (2, 5), 'dropout': (0.0, 0.2, 0.4)}
                # logger.info("parameters:%s"%parameters)
            
            
                raw_args = [
                  '--data_path',data_path,
                  '--dataset_type', dataset_type,
                  '--save_dir', "hyperopt/"+data_name,
                  '--target_columns', target_column,
                  '--smiles_column',smiles_column,
                  '--seed', str(SEED),
                  '--loss_function','mse',
                 # '--checkpoint_dir','ck_dir',
                  '--split_type', 'random', # Literal['random', 'scaffold_balanced', 'predetermined', 'crossval', 'cv', 'cv-no-test', 'index_predetermined', 'random_with_repeated_smiles'] = 'random'
                  '--split_sizes', '0.8', '0.1', '0.1',
                  '--num_folds', str(NUM_FOLDS),
                  '--epochs', '%s'%epochs,
                  '--metric', 'rmse',
                  '--empty_cache',
                  '--extra_metrics', 'r2', 'mae',
                  '--batch_size','%s'%batch_size,
                  '--save_smiles_splits',
                  '--config_save_path',"config/"+data_name,
                  '--num_iter','100',
                  '--empty_cache',
                          '--no_features_scaling',
        '--features_generator','rdkit_2d_normalized',
                  '--search_parameter_keywords','batch_size',
                  '--quiet'] 
    # Hyperopt
    # with patch('sys.argv', raw_args):
                # command_line = ' '.join(raw_args[:])
                # logger.info(f'python hyperparameter_optimization.py {command_line}')

                args = chemprop.args.HyperoptArgs().parse_args(raw_args)
                logger.info("args:%s"%args)
                hyperopt(args=args)

                # for parameter, (min_value, max_value) in parameters.items():
                #     self.assertTrue(min_value <= config[parameter] <= max_value)
    # args = chemprop.args.TrainArgs().parse_args(arguments)
    # mean_score, std_score = chemprop.train.cross_validate(args=args, train_func=chemprop.train.run_training)
    logger.info(raw_args)
    logger.info("data_name:%s data_path:%s target_column:%s"%(data_name,data_path,target_column))
    # logger.info("mean_score:%s std_score:%s"%(mean_score,std_score))

dataset_dir= '/data/project/pbpk/data/'
dataset_name='fup_data_human_2668.csv'
data_path=dataset_dir+dataset_name
logger.info(dataset_dir+dataset_name)
smiles_column='SMILES_STANDARD'
target_column= 'fup_log10'
import datetime
now_time = datetime.datetime.now()
data_str = now_time.strftime('%Y_%m_%d')
 
data_path=dataset_dir+dataset_name
logger.info(dataset_dir+dataset_name)
 
data_name='fup_log10_opt_%s_%s'%(train_param,data_str)
# train(dataset_file,data_name,target_column,smiles_column,batch_size=64,epochs=50)
stopwatch = StopWatch()
run_hyperopt(data_path,data_name,target_column,smiles_column,batch_size=32,epochs=100)
stopwatch.stop('hyper search finish: ')

# python hyperparameter_optimization.py /data/project/pbpk/data/fup_data_human_2668.csv --dataset_type regression --save_dir result/fup_log10 
# --target_columns fup_log10 --smiles_column SMILES --loss_function mse --split_type random --split_sizes 0.8 0.1 0.1 --num_folds 5 --epochs 20 
# --batch_size 32 --save_smiles_splits --config_save_path config/fup_log10 --num_iter 10 --empty_cache --quiet


Overwriting hyperopt/train_fu_rdkit_search.py


In [ ]:
%run hyperopt/train_fu_search.py


In [ ]:
%run hyperopt/train_fu_rdkit_search.py

# train_cl_search

In [2]:
%%writefile hyperopt/train_cl_search.py
import sys
sys.path.append("/data/project/pbpk/chemprop")
import numpy as np
import pandas as pd
from parameterized import parameterized
from io import BytesIO
import json
import os
from tempfile import TemporaryDirectory
from typing import List
import chemprop
from chemprop.constants import TEST_SCORES_FILE_NAME
from chemprop.hyperparameter_optimization import chemprop_hyperopt,hyperopt
from chemprop.interpret import chemprop_interpret
from chemprop.sklearn_predict import sklearn_predict
from chemprop.sklearn_train import sklearn_train
from chemprop.train import chemprop_train, chemprop_predict, evaluate_predictions, chemprop_fingerprint
from chemprop.web.wsgi import build_app
from chemprop.spectra_utils import normalize_spectra, load_phase_mask
from chemprop.features import load_features

from io import BytesIO
import json
import os
from tempfile import TemporaryDirectory
from typing import List
import unittest
from unittest import TestCase
from unittest.mock import patch
from chemprop.utils import StopWatch
from loguru import logger
SEED = 2023
EPOCHS = 10
NUM_FOLDS = 5
NUM_ITER = 2
SIZE = 10
DEPTH = 3
DELTA = 0.025
train_param=""
def run_hyperopt(data_path,data_name,target_column,smiles_column,batch_size=64,epochs=100):
    flags: List[str] = None
    with TemporaryDirectory() as save_dir:
                # Train
                config_save_path = os.path.join(save_dir, 'config.json')
                dataset_type="regression"
             
                # Check results

                # parameters = {'epochs':{50,100,200},'batch_size':{16,32,64,128,256,512,1024},'depth': (2, 6), 'hidden_size': (300, 2400), 'ffn_hidden_size': (300, 2400), 'ffn_num_layers': (1, 3), 'dropout': (0.0, 0.4)}
            
                # parameters = {'depth': (2, 6), 'hidden_size': (300, 2400), 'ffn_hidden_size': (300, 2400), 'ffn_num_layers': (1, 3), 'dropout': (0.0, 0.4)}
                # parameters = {'epochs':{50,100,200},'batch_size':{16,32,64,128,256,512,1024},'depth': (2, 6), 'hidden_size': (300, 1200),
                #               'ffn_hidden_size': (300, 1200), 'ffn_num_layers': (1, 3), 'dropout': (0.0,0.2, 0.4)}
                # parameters = {'epochs':{50,100,200},'batch_size':{16,32,64,128,256,512,1024},'depth': (2, 5), 'dropout': (0.0, 0.2, 0.4)}
                # parameters = {'batch_size':{16,32,64,128,256,512,1024},'depth': (2, 5), 'dropout': (0.0, 0.2, 0.4)}
                # parameters = {'batch_size':{16,32,64,128,256,512,1024},'depth': (2, 5), 'dropout': (0.0, 0.2, 0.4)}
                # logger.info("parameters:%s"%parameters)
            
            
                raw_args = [
                  '--data_path',data_path,
                  '--dataset_type', dataset_type,
                  '--save_dir', "hyperopt/"+data_name,
                  '--target_columns', target_column,
                  '--smiles_column',smiles_column,
                  '--seed', str(SEED),
                  '--loss_function','mse',
                 # '--checkpoint_dir','ck_dir',
                  '--split_type', 'random', # Literal['random', 'scaffold_balanced', 'predetermined', 'crossval', 'cv', 'cv-no-test', 'index_predetermined', 'random_with_repeated_smiles'] = 'random'
                  '--split_sizes', '0.8', '0.1', '0.1',
                  '--num_folds', str(NUM_FOLDS),
                  '--epochs', '%s'%epochs,
                  '--metric', 'rmse',
                  '--empty_cache',
                  '--extra_metrics', 'r2', 'mae',
                  '--batch_size','%s'%batch_size,
                  '--save_smiles_splits',
                  '--config_save_path',"config/"+data_name,
                  '--num_iter','50',
                  '--gpu','1',
                  '--empty_cache',
                  '--search_parameter_keywords','batch_size',
                  '--quiet']
                
    # Hyperopt
    # with patch('sys.argv', raw_args):
                # command_line = ' '.join(raw_args[:])
                # logger.info(f'python hyperparameter_optimization.py {command_line}')

                args = chemprop.args.HyperoptArgs().parse_args(raw_args)
                logger.info("args:%s"%args)
                hyperopt(args=args)

                # for parameter, (min_value, max_value) in parameters.items():
                #     self.assertTrue(min_value <= config[parameter] <= max_value)
    # args = chemprop.args.TrainArgs().parse_args(arguments)
    # mean_score, std_score = chemprop.train.cross_validate(args=args, train_func=chemprop.train.run_training)
    logger.info(raw_args)
    logger.info("data_name:%s data_path:%s target_column:%s"%(data_name,data_path,target_column))
    # logger.info("mean_score:%s std_score:%s"%(mean_score,std_score))

import datetime
now_time = datetime.datetime.now()
data_str = now_time.strftime('%Y_%m_%d')
 
dataset_dir= '/data/project/pbpk/data/'
dataset_name='pk_cl_1215_train_2023_v1.csv'
data_path=dataset_dir+dataset_name
logger.info(dataset_dir+dataset_name)
smiles_column='SMILES_STANDARD'
target_column= 'human CL (mL/min/kg)_log10'
# target_column= 'human CL (mL/min/kg)_log2'
# target_column= 'Y'
data_name='pk_cl_1215_train_opt_log10_32_100_%s_%s'%(train_param,data_str)
logger.info("data_name:%s"%data_name)
# train(dataset_file,data_name,target_column,smiles_column,batch_size=64,epochs=50)
run_hyperopt(data_path,data_name,target_column,smiles_column,batch_size=32,epochs=100)

# python hyperparameter_optimization.py /data/project/pbpk/data/fup_data_human_2668.csv --dataset_type regression --save_dir result/fup_log10 
# --target_columns fup_log10 --smiles_column SMILES --loss_function mse --split_type random --split_sizes 0.8 0.1 0.1 --num_folds 5 --epochs 20 
# --batch_size 32 --save_smiles_splits --config_save_path config/fup_log10 --num_iter 10 --empty_cache --quiet


Overwriting hyperopt/train_cl_search.py


## train_cl_rdkit_search

In [1]:
%%writefile hyperopt/train_cl_rdkit_a_search.py
import sys
sys.path.append("/data/project/pbpk/chemprop")
import numpy as np
import pandas as pd
# from parameterized import parameterized
from io import BytesIO
import json
import os
from tempfile import TemporaryDirectory
from typing import List
import chemprop
from chemprop.constants import TEST_SCORES_FILE_NAME
from chemprop.hyperparameter_optimization import chemprop_hyperopt,hyperopt
from chemprop.interpret import chemprop_interpret
from chemprop.sklearn_predict import sklearn_predict
from chemprop.sklearn_train import sklearn_train
from chemprop.train import chemprop_train, chemprop_predict, evaluate_predictions, chemprop_fingerprint
from chemprop.web.wsgi import build_app
from chemprop.spectra_utils import normalize_spectra, load_phase_mask
from chemprop.features import load_features
import unittest
from unittest import TestCase
from unittest.mock import patch
from chemprop.utils import StopWatch
from loguru import logger
import os,sys
import time
from pathlib import Path
project_path = Path.cwd().parent
log_path = Path(project_path, "log")
log_time = time.strftime("%Y_%m_%d")
logger.remove()
logger.add(f'logs/cl_train_epoch_opt_{log_time}.log', format='{time:YYYY-MM-DD HH:mm:ss} | ' 
                                "{file} | "# 
                               "{process.name} | "  # 进程名
                               "{thread.name} | "  # 进程名
                               '{module}.{function}:{line} - {level} -{message}',level="INFO", enqueue=True,
encoding="utf-8", rotation="12:00"
)
logger.add(sys.stderr, format='{time:YYYY-MM-DD HH:mm:ss} | ' 
                                "{file} | "# 
                               "{process.name} | "  # 进程名
                               "{thread.name} | "  # 进程名
                               '{module}.{function}:{line} - {level} -{message}',level="INFO", enqueue=True
)
SEED = 2023
EPOCHS = 64
NUM_FOLDS = 5
NUM_ITER = 2
SIZE = 10
DEPTH = 3
DELTA = 0.025
train_param="rdkit_2d_normalized"
       
def run_hyperopt(data_path,data_name,target_column,smiles_column,batch_size=64,epochs=100):
    flags: List[str] = None
    with TemporaryDirectory() as save_dir:
                # Train
                config_save_path = os.path.join(save_dir, 'config.json')
                dataset_type="regression"
             
                # Check results

                # parameters = {'epochs':{50,100,200},'batch_size':{16,32,64,128,256,512,1024},'depth': (2, 6), 'hidden_size': (300, 2400), 'ffn_hidden_size': (300, 2400), 'ffn_num_layers': (1, 3), 'dropout': (0.0, 0.4)}
            
                # parameters = {'depth': (2, 6), 'hidden_size': (300, 2400), 'ffn_hidden_size': (300, 2400), 'ffn_num_layers': (1, 3), 'dropout': (0.0, 0.4)}
                # parameters = {'epochs':{50,100,200},'batch_size':{16,32,64,128,256,512,1024},'depth': (2, 6), 'hidden_size': (300, 1200),
                #               'ffn_hidden_size': (300, 1200), 'ffn_num_layers': (1, 3), 'dropout': (0.0,0.2, 0.4)}
                # parameters = {'epochs':{50,100,200},'batch_size':{16,32,64,128,256,512,1024},'depth': (2, 5), 'dropout': (0.0, 0.2, 0.4)}
                # parameters = {'batch_size':{16,32,64,128,256,512,1024},'depth': (2, 5), 'dropout': (0.0, 0.2, 0.4)}
                # parameters = {'batch_size':{16,32,64,128,256,512,1024},'depth': (2, 5), 'dropout': (0.0, 0.2, 0.4)}
                # logger.info("parameters:%s"%parameters)
            
            
                raw_args = [
                  '--data_path',data_path,
                  '--dataset_type', dataset_type,
                  '--save_dir', "hyperopt/"+data_name,
                  '--target_columns', target_column,
                  '--smiles_column',smiles_column,
                  '--seed', str(SEED),
                  '--loss_function','mse',
                 # '--checkpoint_dir','ck_dir',
                  '--split_type', 'random', # Literal['random', 'scaffold_balanced', 'predetermined', 'crossval', 'cv', 'cv-no-test', 'index_predetermined', 'random_with_repeated_smiles'] = 'random'
                  '--split_sizes', '0.8', '0.1', '0.1',
                  '--num_folds', str(NUM_FOLDS),
                  '--epochs', '%s'%epochs,
                  '--metric', 'rmse',
                  '--empty_cache',
                  '--extra_metrics', 'r2', 'mae',
                  '--batch_size','%s'%batch_size,
                  '--save_smiles_splits',
                  '--config_save_path',"config/"+data_name,
                  '--num_iter','60',
                  '--empty_cache',
                  '--num_workers','32',
                  '--gpu','1',
                  '--no_features_scaling',
                  '--features_generator','rdkit_2d_normalized',
                  '--search_parameter_keywords','basic','batch_size',
                  '--quiet']
        #           "activation": hp.choice(
        #     "activation", options=["ReLU", "LeakyReLU", "PReLU", "tanh", "SELU", "ELU"]
        # ),
    #     "aggregation": hp.choice("aggregation", options=["mean", "sum", "norm"]),
    #     "aggregation_norm": hp.quniform("aggregation_norm", low=1, high=200, q=1),
    #     "batch_size": hp.quniform("batch_size", low=5, high=200, q=5),
    #     "depth": hp.quniform("depth", low=2, high=6, q=1),
    #     "dropout": hp.quniform("dropout", low=0.0, high=0.4, q=0.05),
    #     "ffn_hidden_size": hp.quniform("ffn_hidden_size", low=300, high=2400, q=100),
    #     "ffn_num_layers": hp.quniform("ffn_num_layers", low=1, high=3, q=1),
    #     "final_lr_ratio": hp.loguniform("final_lr_ratio", low=np.log(1e-4), high=0.),
    #     "hidden_size": hp.quniform("hidden_size", low=300, high=2400, q=100),
    #     "init_lr_ratio": hp.loguniform("init_lr_ratio", low=np.log(1e-4), high=0.),
    #     "linked_hidden_size": hp.quniform("linked_hidden_size", low=300, high=2400, q=100),
    #     "max_lr": hp.loguniform("max_lr", low=np.log(1e-6), high=np.log(1e-2)),
    #     "warmup_epochs": hp.quniform("warmup_epochs", low=1, high=train_epochs // 2, q=1)
    # # Hyperopt
    # with patch('sys.argv', raw_args):
                # command_line = ' '.join(raw_args[:])
                # logger.info(f'python hyperparameter_optimization.py {command_line}')

                args = chemprop.args.HyperoptArgs().parse_args(raw_args)
                logger.info("args:%s"%args)
                hyperopt(args=args)

                # for parameter, (min_value, max_value) in parameters.items():
                #     self.assertTrue(min_value <= config[parameter] <= max_value)
    # args = chemprop.args.TrainArgs().parse_args(arguments)
    # mean_score, std_score = chemprop.train.cross_validate(args=args, train_func=chemprop.train.run_training)
    logger.info(raw_args)
    logger.info("data_name:%s data_path:%s target_column:%s"%(data_name,data_path,target_column))
    # logger.info("mean_score:%s std_score:%s"%(mean_score,std_score))

import datetime
now_time = datetime.datetime.now()
data_str = now_time.strftime('%Y_%m_%d')
 
dataset_dir= '/data/project/pbpk/data/'
dataset_name='pk_cl_1215_train_2023_v1.csv'
data_path=dataset_dir+dataset_name
logger.info(dataset_dir+dataset_name)
smiles_column='SMILES_STANDARD'
target_column= 'human CL (mL/min/kg)_log10'
# target_column= 'human CL (mL/min/kg)_log2'
# target_column= 'Y'
data_name='cl_1215_basic_%s_%s'%(train_param,data_str)
logger.info("data_name:%s"%data_name)
# train(dataset_file,data_name,target_column,smiles_column,batch_size=64,epochs=50)
stopwatch = StopWatch()
      
        
run_hyperopt(data_path,data_name,target_column,smiles_column,batch_size=32,epochs=100)
stopwatch.stop('hyper search finish: ')

# python hyperparameter_optimization.py /data/project/pbpk/data/fup_data_human_2668.csv --dataset_type regression --save_dir result/fup_log10 
# --target_columns fup_log10 --smiles_column SMILES --loss_function mse --split_type random --split_sizes 0.8 0.1 0.1 --num_folds 5 --epochs 20 
# --batch_size 32 --save_smiles_splits --config_save_path config/fup_log10 --num_iter 10 --empty_cache --quiet
## #/home/atest/anaconda3/envs/pkmodel/bin/python train_cl_rdkit_search.py

Overwriting hyperopt/train_cl_rdkit_a_search.py


## train_cl_morgan_search

In [2]:
%%writefile hyperopt/train_cl_morgan_search.py
import sys
sys.path.append("/data/project/pbpk/chemprop")
import numpy as np
import pandas as pd
# from parameterized import parameterized
from io import BytesIO
import json
import os
from tempfile import TemporaryDirectory
from typing import List
import chemprop
from chemprop.constants import TEST_SCORES_FILE_NAME
from chemprop.hyperparameter_optimization import chemprop_hyperopt,hyperopt
from chemprop.interpret import chemprop_interpret
from chemprop.sklearn_predict import sklearn_predict
from chemprop.sklearn_train import sklearn_train
from chemprop.train import chemprop_train, chemprop_predict, evaluate_predictions, chemprop_fingerprint
from chemprop.web.wsgi import build_app
from chemprop.spectra_utils import normalize_spectra, load_phase_mask
from chemprop.features import load_features
import unittest
from unittest import TestCase
from unittest.mock import patch
from chemprop.utils import StopWatch
from loguru import logger
import os,sys
import time
from pathlib import Path
project_path = Path.cwd().parent
log_path = Path(project_path, "log")
log_time = time.strftime("%Y_%m_%d")
logger.remove()
logger.add(f'logs/cl_train_opt_{log_time}.log', format='{time:YYYY-MM-DD HH:mm:ss} | ' 
                                "{file} | "# 
                               "{process.name} | "  # 进程名
                               "{thread.name} | "  # 进程名
                               '{module}.{function}:{line} - {level} -{message}',level="INFO", enqueue=True,
encoding="utf-8", rotation="12:00"
)
logger.add(sys.stderr, format='{time:YYYY-MM-DD HH:mm:ss} | ' 
                                "{file} | "# 
                               "{process.name} | "  # 进程名
                               "{thread.name} | "  # 进程名
                               '{module}.{function}:{line} - {level} -{message}',level="INFO", enqueue=True
)
SEED = 2023
EPOCHS = 64
NUM_FOLDS = 5
NUM_ITER = 50
SIZE = 10
DEPTH = 3
DELTA = 0.025
train_param="morgan"
       
def run_hyperopt(data_path,data_name,target_column,smiles_column,batch_size=64,epochs=100):
    flags: List[str] = None
    with TemporaryDirectory() as save_dir:
                # Train
                config_save_path = os.path.join(save_dir, 'config.json')
                dataset_type="regression"
             
                # Check results

                # parameters = {'epochs':{50,100,200},'batch_size':{16,32,64,128,256,512,1024},'depth': (2, 6), 'hidden_size': (300, 2400), 'ffn_hidden_size': (300, 2400), 'ffn_num_layers': (1, 3), 'dropout': (0.0, 0.4)}
            
                # parameters = {'depth': (2, 6), 'hidden_size': (300, 2400), 'ffn_hidden_size': (300, 2400), 'ffn_num_layers': (1, 3), 'dropout': (0.0, 0.4)}
                # parameters = {'epochs':{50,100,200},'batch_size':{16,32,64,128,256,512,1024},'depth': (2, 6), 'hidden_size': (300, 1200),
                #               'ffn_hidden_size': (300, 1200), 'ffn_num_layers': (1, 3), 'dropout': (0.0,0.2, 0.4)}
                # parameters = {'epochs':{50,100,200},'batch_size':{16,32,64,128,256,512,1024},'depth': (2, 5), 'dropout': (0.0, 0.2, 0.4)}
                # parameters = {'batch_size':{16,32,64,128,256,512,1024},'depth': (2, 5), 'dropout': (0.0, 0.2, 0.4)}
                # parameters = {'batch_size':{16,32,64,128,256,512,1024},'depth': (2, 5), 'dropout': (0.0, 0.2, 0.4)}
                # logger.info("parameters:%s"%parameters)
            
            
                raw_args = [
                  '--data_path',data_path,
                  '--dataset_type', dataset_type,
                  '--save_dir', "hyperopt/"+data_name,
                  '--target_columns', target_column,
                  '--smiles_column',smiles_column,
                  '--seed', str(SEED),
                  '--loss_function','mse',
                 # '--checkpoint_dir','ck_dir',
                  '--split_type', 'random', # Literal['random', 'scaffold_balanced', 'predetermined', 'crossval', 'cv', 'cv-no-test', 'index_predetermined', 'random_with_repeated_smiles'] = 'random'
                  '--split_sizes', '0.8', '0.1', '0.1',
                  '--num_folds', str(NUM_FOLDS),
                  '--epochs', '%s'%epochs,
                  '--metric', 'rmse',
                  '--empty_cache',
                  '--extra_metrics', 'r2', 'mae',
                  '--batch_size','%s'%batch_size,
                  '--save_smiles_splits',
                  '--config_save_path',"config/"+data_name,
                  '--num_iter','50',
                  '--empty_cache',
                  '--num_workers','32',
                  '--gpu','0',
                  # '--no_features_scaling',
                  '--features_generator','morgan',
                 '--search_parameter_keywords','basic','batch_size','activation','aggregation',
                  '--quiet']
        #           "activation": hp.choice(
        #     "activation", options=["ReLU", "LeakyReLU", "PReLU", "tanh", "SELU", "ELU"]
        # ),
    #     "aggregation": hp.choice("aggregation", options=["mean", "sum", "norm"]),
    #     "aggregation_norm": hp.quniform("aggregation_norm", low=1, high=200, q=1),
    #     "batch_size": hp.quniform("batch_size", low=5, high=200, q=5),
    #     "depth": hp.quniform("depth", low=2, high=6, q=1),
    #     "dropout": hp.quniform("dropout", low=0.0, high=0.4, q=0.05),
    #     "ffn_hidden_size": hp.quniform("ffn_hidden_size", low=300, high=2400, q=100),
    #     "ffn_num_layers": hp.quniform("ffn_num_layers", low=1, high=3, q=1),
    #     "final_lr_ratio": hp.loguniform("final_lr_ratio", low=np.log(1e-4), high=0.),
    #     "hidden_size": hp.quniform("hidden_size", low=300, high=2400, q=100),
    #     "init_lr_ratio": hp.loguniform("init_lr_ratio", low=np.log(1e-4), high=0.),
    #     "linked_hidden_size": hp.quniform("linked_hidden_size", low=300, high=2400, q=100),
    #     "max_lr": hp.loguniform("max_lr", low=np.log(1e-6), high=np.log(1e-2)),
    #     "warmup_epochs": hp.quniform("warmup_epochs", low=1, high=train_epochs // 2, q=1)
    # # Hyperopt
    # with patch('sys.argv', raw_args):
                # command_line = ' '.join(raw_args[:])
                # logger.info(f'python hyperparameter_optimization.py {command_line}')

                args = chemprop.args.HyperoptArgs().parse_args(raw_args)
                logger.info("args:%s"%args)
                hyperopt(args=args)

                # for parameter, (min_value, max_value) in parameters.items():
                #     self.assertTrue(min_value <= config[parameter] <= max_value)
    # args = chemprop.args.TrainArgs().parse_args(arguments)
    # mean_score, std_score = chemprop.train.cross_validate(args=args, train_func=chemprop.train.run_training)
    logger.info(raw_args)
    logger.info("data_name:%s data_path:%s target_column:%s"%(data_name,data_path,target_column))
    # logger.info("mean_score:%s std_score:%s"%(mean_score,std_score))

import datetime
now_time = datetime.datetime.now()
data_str = now_time.strftime('%Y_%m_%d')
 
dataset_dir= '/data/project/pbpk/data/'
dataset_name='pk_cl_1215_train_2023.csv'
data_path=dataset_dir+dataset_name
logger.info(dataset_dir+dataset_name)
smiles_column='SMILES'
target_column= 'human CL (mL/min/kg)_log10'
# target_column= 'human CL (mL/min/kg)_log2'
# target_column= 'Y'
data_name='pk_cl_1215_log10_16_100_%s_%s'%(train_param,data_str)
logger.info("data_name:%s"%data_name)
# train(dataset_file,data_name,target_column,smiles_column,batch_size=64,epochs=50)
stopwatch = StopWatch()
      
        
run_hyperopt(data_path,data_name,target_column,smiles_column,batch_size=16,epochs=100)
stopwatch.stop('hyper search finish: ')

# python hyperparameter_optimization.py /data/project/pbpk/data/fup_data_human_2668.csv --dataset_type regression --save_dir result/fup_log10 
# --target_columns fup_log10 --smiles_column SMILES --loss_function mse --split_type random --split_sizes 0.8 0.1 0.1 --num_folds 5 --epochs 20 
# --batch_size 32 --save_smiles_splits --config_save_path config/fup_log10 --num_iter 10 --empty_cache --quiet


Overwriting hyperopt/train_cl_morgan_search.py


In [ ]:
%run hyperopt/train_cl_search.py

In [ ]:
%run hyperopt/train_cl_rdkit_search.py
#conda activate pkmodel;python train_cl_rdkit_search.py

# train_caco2_search

In [5]:
%%writefile hyperopt/train_caco2_search.py
import sys
sys.path.append("/data/project/pbpk/chemprop")
import numpy as np
import pandas as pd
from parameterized import parameterized
from io import BytesIO
import json
import os
from chemprop.utils import StopWatch
from tempfile import TemporaryDirectory
from typing import List
import chemprop
from chemprop.constants import TEST_SCORES_FILE_NAME
from chemprop.hyperparameter_optimization import chemprop_hyperopt,hyperopt
from chemprop.interpret import chemprop_interpret
from chemprop.sklearn_predict import sklearn_predict
from chemprop.sklearn_train import sklearn_train
from chemprop.train import chemprop_train, chemprop_predict, evaluate_predictions, chemprop_fingerprint
from chemprop.web.wsgi import build_app
from chemprop.spectra_utils import normalize_spectra, load_phase_mask
from chemprop.features import load_features

from io import BytesIO
import json
import os
from tempfile import TemporaryDirectory
from typing import List
import unittest
from unittest import TestCase
from unittest.mock import patch
 
from loguru import logger
import os,sys
import time
from pathlib import Path
project_path = Path.cwd().parent
log_path = Path(project_path, "log")
log_time = time.strftime("%Y_%m_%d")
logger.remove()
logger.add(f'logs/caco2_train_ori_opt_{log_time}.log', format='{time:YYYY-MM-DD HH:mm:ss} | ' 
                                "{file} | "# 
                               "{process.name} | "  # 进程名
                               "{thread.name} | "  # 进程名
                               '{module}.{function}:{line} - {level} -{message}',level="INFO", enqueue=True,
encoding="utf-8", rotation="12:00"
)
logger.add(sys.stderr, format='{time:YYYY-MM-DD HH:mm:ss} | ' 
                                "{file} | "# 
                               "{process.name} | "  # 进程名
                               "{thread.name} | "  # 进程名
                               '{module}.{function}:{line} - {level} -{message}',level="INFO", enqueue=True
)
SEED = 2023
EPOCHS = 10
NUM_FOLDS = 5
NUM_ITER = 2
SIZE = 10
DEPTH = 3
DELTA = 0.025
train_param=""
def run_hyperopt(data_path,data_name,target_column,smiles_column,batch_size=64,epochs=100):
    flags: List[str] = None
    with TemporaryDirectory() as save_dir:
                # Train
                config_save_path = os.path.join(save_dir, 'config.json')
                dataset_type="regression"
             
                # Check results

                # parameters = {'epochs':{50,100,200},'batch_size':{16,32,64,128,256,512,1024},'depth': (2, 6), 'hidden_size': (300, 2400), 'ffn_hidden_size': (300, 2400), 'ffn_num_layers': (1, 3), 'dropout': (0.0, 0.4)}
            
                # parameters = {'depth': (2, 6), 'hidden_size': (300, 2400), 'ffn_hidden_size': (300, 2400), 'ffn_num_layers': (1, 3), 'dropout': (0.0, 0.4)}
                # parameters = {'epochs':{50,100,200},'batch_size':{16,32,64,128,256,512,1024},'depth': (2, 6), 'hidden_size': (300, 1200),
                #               'ffn_hidden_size': (300, 1200), 'ffn_num_layers': (1, 3), 'dropout': (0.0,0.2, 0.4)}
                # parameters = {'epochs':{50,100,200},'batch_size':{16,32,64,128,256,512,1024},'depth': (2, 5), 'dropout': (0.0, 0.2, 0.4)}
                # parameters = {'batch_size':{16,32,64,128,256,512,1024},'depth': (2, 5), 'dropout': (0.0, 0.2, 0.4)}
                # parameters = {'batch_size':{16,32,64,128,256,512,1024},'depth': (2, 5), 'dropout': (0.0, 0.2, 0.4)}
                # logger.info("parameters:%s"%parameters)
            
            
                raw_args = [
                  '--data_path',data_path,
                  '--dataset_type', dataset_type,
                  '--save_dir', "hyperopt/"+data_name,
                  '--target_columns', target_column,
                  '--smiles_column',smiles_column,
                  '--seed', str(SEED),
                  '--loss_function','mse',
                 # '--checkpoint_dir','ck_dir',
                  '--split_type', 'random', # Literal['random', 'scaffold_balanced', 'predetermined', 'crossval', 'cv', 'cv-no-test', 'index_predetermined', 'random_with_repeated_smiles'] = 'random'
                  '--split_sizes', '0.8', '0.1', '0.1',
                  '--num_folds', str(NUM_FOLDS),
                  '--epochs', '%s'%epochs,
                  '--metric', 'rmse',
                  '--empty_cache',
                  '--extra_metrics', 'r2', 'mae',
                  '--batch_size','%s'%batch_size,
                  '--save_smiles_splits',
                  '--config_save_path',"config/"+data_name,
                 '--num_iter','60',
                  '--empty_cache',
                  '--num_workers','8',
                  '--gpu','0',
                  '--search_parameter_keywords','batch_size',
                  '--quiet']
    # Hyperopt
    # with patch('sys.argv', raw_args):
                # command_line = ' '.join(raw_args[:])
                # logger.info(f'python hyperparameter_optimization.py {command_line}')

                args = chemprop.args.HyperoptArgs().parse_args(raw_args)
                logger.info("args:%s"%args)
                hyperopt(args=args)

                # for parameter, (min_value, max_value) in parameters.items():
                #     self.assertTrue(min_value <= config[parameter] <= max_value)
    # args = chemprop.args.TrainArgs().parse_args(arguments)
    # mean_score, std_score = chemprop.train.cross_validate(args=args, train_func=chemprop.train.run_training)
    logger.info(raw_args)
    logger.info("data_name:%s data_path:%s target_column:%s"%(data_name,data_path,target_column))
    # logger.info("mean_score:%s std_score:%s"%(mean_score,std_score))

import datetime
now_time = datetime.datetime.now()
data_str = now_time.strftime('%Y_%m_%d')
dataset_dir= '/data/project/pbpk/data/'
dataset_name='caco2_train_set_6083.csv'
data_path=dataset_dir+dataset_name
logger.info(dataset_dir+dataset_name)
smiles_column='SMILES_STANDARD'
# target_column= 'human CL (mL/min/kg)_log10'
target_column= 'caco2_log10'
# target_column= 'Y'
data_name='caco2_log10_ori_%s_%s'%(train_param,data_str)
logger.info("data_name:%s"%data_name)
run_hyperopt(data_path,data_name,target_column,smiles_column,batch_size=32,epochs=100)
# python hyperparameter_optimization.py /data/project/pbpk/data/fup_data_human_2668.csv --dataset_type regression --save_dir result/fup_log10 
# --target_columns fup_log10 --smiles_column SMILES --loss_function mse --split_type random --split_sizes 0.8 0.1 0.1 --num_folds 5 --epochs 20 
# --batch_size 32 --save_smiles_splits --config_save_path config/fup_log10 --num_iter 10 --empty_cache --quiet


Overwriting hyperopt/train_caco2_search.py


## train_caco2_rdkit_search

In [5]:
%%writefile hyperopt/train_caco2_rd_search.py
import sys
sys.path.append("/data/project/pbpk/chemprop")
import numpy as np
import pandas as pd
from parameterized import parameterized
from io import BytesIO
import json
import os
from chemprop.utils import StopWatch
from tempfile import TemporaryDirectory
from typing import List
import chemprop
from chemprop.constants import TEST_SCORES_FILE_NAME
from chemprop.hyperparameter_optimization import chemprop_hyperopt,hyperopt
from chemprop.interpret import chemprop_interpret
from chemprop.sklearn_predict import sklearn_predict
from chemprop.sklearn_train import sklearn_train
from chemprop.train import chemprop_train, chemprop_predict, evaluate_predictions, chemprop_fingerprint
from chemprop.web.wsgi import build_app
from chemprop.spectra_utils import normalize_spectra, load_phase_mask
from chemprop.features import load_features

from io import BytesIO
import json
import os
from tempfile import TemporaryDirectory
from typing import List
import unittest
from unittest import TestCase
from unittest.mock import patch
 
from loguru import logger
import os,sys
import time
from pathlib import Path
project_path = Path.cwd().parent
log_path = Path(project_path, "log")
log_time = time.strftime("%Y_%m_%d")
logger.remove()
logger.add(f'logs/caco2_train_opt_{log_time}.log', format='{time:YYYY-MM-DD HH:mm:ss} | ' 
                                "{file} | "# 
                               "{process.name} | "  # 进程名
                               "{thread.name} | "  # 进程名
                               '{module}.{function}:{line} - {level} -{message}',level="INFO", enqueue=True,
encoding="utf-8", rotation="12:00"
)
logger.add(sys.stderr, format='{time:YYYY-MM-DD HH:mm:ss} | ' 
                                "{file} | "# 
                               "{process.name} | "  # 进程名
                               "{thread.name} | "  # 进程名
                               '{module}.{function}:{line} - {level} -{message}',level="INFO", enqueue=True
)
SEED = 2023
EPOCHS = 10
NUM_FOLDS = 5
NUM_ITER = 2
SIZE = 10
DEPTH = 3
DELTA = 0.025
train_param=""
def run_hyperopt(data_path,data_name,target_column,smiles_column,batch_size=64,epochs=100):
    flags: List[str] = None
    with TemporaryDirectory() as save_dir:
                # Train
                config_save_path = os.path.join(save_dir, 'config.json')
                dataset_type="regression"
             
                # Check results

                # parameters = {'epochs':{50,100,200},'batch_size':{16,32,64,128,256,512,1024},'depth': (2, 6), 'hidden_size': (300, 2400), 'ffn_hidden_size': (300, 2400), 'ffn_num_layers': (1, 3), 'dropout': (0.0, 0.4)}
            
                # parameters = {'depth': (2, 6), 'hidden_size': (300, 2400), 'ffn_hidden_size': (300, 2400), 'ffn_num_layers': (1, 3), 'dropout': (0.0, 0.4)}
                # parameters = {'epochs':{50,100,200},'batch_size':{16,32,64,128,256,512,1024},'depth': (2, 6), 'hidden_size': (300, 1200),
                #               'ffn_hidden_size': (300, 1200), 'ffn_num_layers': (1, 3), 'dropout': (0.0,0.2, 0.4)}
                # parameters = {'epochs':{50,100,200},'batch_size':{16,32,64,128,256,512,1024},'depth': (2, 5), 'dropout': (0.0, 0.2, 0.4)}
                # parameters = {'batch_size':{16,32,64,128,256,512,1024},'depth': (2, 5), 'dropout': (0.0, 0.2, 0.4)}
                # parameters = {'batch_size':{16,32,64,128,256,512,1024},'depth': (2, 5), 'dropout': (0.0, 0.2, 0.4)}
                # logger.info("parameters:%s"%parameters)
            
            
                raw_args = [
                  '--data_path',data_path,
                  '--dataset_type', dataset_type,
                  '--save_dir', "hyperopt/"+data_name,
                  '--target_columns', target_column,
                  '--smiles_column',smiles_column,
                  '--seed', str(SEED),
                  '--loss_function','mse',
                 # '--checkpoint_dir','ck_dir',
                  '--split_type', 'random', # Literal['random', 'scaffold_balanced', 'predetermined', 'crossval', 'cv', 'cv-no-test', 'index_predetermined', 'random_with_repeated_smiles'] = 'random'
                  '--split_sizes', '0.8', '0.1', '0.1',
                  '--num_folds', str(NUM_FOLDS),
                  '--epochs', '%s'%epochs,
                  '--metric', 'rmse',
                  '--empty_cache',
                  '--extra_metrics', 'r2', 'mae',
                  '--batch_size','%s'%batch_size,
                  '--save_smiles_splits',
                  '--config_save_path',"config/"+data_name,
                 '--num_iter','60',
                  '--empty_cache',
                  '--num_workers','8',
                  '--gpu','0',
                  '--no_features_scaling',
                  '--features_generator','rdkit_2d_normalized',
                  '--search_parameter_keywords','batch_size',
                  '--quiet']
    # Hyperopt
    # with patch('sys.argv', raw_args):
                # command_line = ' '.join(raw_args[:])
                # logger.info(f'python hyperparameter_optimization.py {command_line}')

                args = chemprop.args.HyperoptArgs().parse_args(raw_args)
                logger.info("args:%s"%args)
                hyperopt(args=args)

                # for parameter, (min_value, max_value) in parameters.items():
                #     self.assertTrue(min_value <= config[parameter] <= max_value)
    # args = chemprop.args.TrainArgs().parse_args(arguments)
    # mean_score, std_score = chemprop.train.cross_validate(args=args, train_func=chemprop.train.run_training)
    logger.info(raw_args)
    logger.info("data_name:%s data_path:%s target_column:%s"%(data_name,data_path,target_column))
    # logger.info("mean_score:%s std_score:%s"%(mean_score,std_score))

import datetime
now_time = datetime.datetime.now()
data_str = now_time.strftime('%Y_%m_%d')
dataset_dir= '/data/project/pbpk/data/'
dataset_name='caco2_train_set_6083.csv'
data_path=dataset_dir+dataset_name
logger.info(dataset_dir+dataset_name)
smiles_column='SMILES_STANDARD'
# target_column= 'human CL (mL/min/kg)_log10'
target_column= 'caco2_log10'
# target_column= 'Y'
data_name='caco2_log10_%s_%s'%(train_param,data_str)
logger.info("data_name:%s"%data_name)
run_hyperopt(data_path,data_name,target_column,smiles_column,batch_size=32,epochs=100)
# python hyperparameter_optimization.py /data/project/pbpk/data/fup_data_human_2668.csv --dataset_type regression --save_dir result/fup_log10 
# --target_columns fup_log10 --smiles_column SMILES --loss_function mse --split_type random --split_sizes 0.8 0.1 0.1 --num_folds 5 --epochs 20 
# --batch_size 32 --save_smiles_splits --config_save_path config/fup_log10 --num_iter 10 --empty_cache --quiet


Overwriting hyperopt/train_caco2_rdkit_search.py


## train_caco2_morgan_search

In [6]:
%%writefile hyperopt/train_caco2_morgan_search.py
import sys
sys.path.append("/data/project/pbpk/chemprop")
import numpy as np
import pandas as pd
# from parameterized import parameterized
from io import BytesIO
import json
import os
from chemprop.utils import StopWatch
from tempfile import TemporaryDirectory
from typing import List
import chemprop
from chemprop.constants import TEST_SCORES_FILE_NAME
from chemprop.hyperparameter_optimization import chemprop_hyperopt,hyperopt
from chemprop.interpret import chemprop_interpret
from chemprop.sklearn_predict import sklearn_predict
from chemprop.sklearn_train import sklearn_train
from chemprop.train import chemprop_train, chemprop_predict, evaluate_predictions, chemprop_fingerprint
from chemprop.web.wsgi import build_app
from chemprop.spectra_utils import normalize_spectra, load_phase_mask
from chemprop.features import load_features

from io import BytesIO
import json
import os
from tempfile import TemporaryDirectory
from typing import List
import unittest
from unittest import TestCase
from unittest.mock import patch
 
from loguru import logger
SEED = 2023
EPOCHS = 10
NUM_FOLDS = 5
NUM_ITER = 2
SIZE = 10
DEPTH = 3
DELTA = 0.025
train_param=""
def run_hyperopt(data_path,data_name,target_column,smiles_column,batch_size=64,epochs=100):
    flags: List[str] = None
    with TemporaryDirectory() as save_dir:
                # Train
                config_save_path = os.path.join(save_dir, 'config.json')
                dataset_type="regression"
             
                # Check results

                # parameters = {'epochs':{50,100,200},'batch_size':{16,32,64,128,256,512,1024},'depth': (2, 6), 'hidden_size': (300, 2400), 'ffn_hidden_size': (300, 2400), 'ffn_num_layers': (1, 3), 'dropout': (0.0, 0.4)}
            
                # parameters = {'depth': (2, 6), 'hidden_size': (300, 2400), 'ffn_hidden_size': (300, 2400), 'ffn_num_layers': (1, 3), 'dropout': (0.0, 0.4)}
                # parameters = {'epochs':{50,100,200},'batch_size':{16,32,64,128,256,512,1024},'depth': (2, 6), 'hidden_size': (300, 1200),
                #               'ffn_hidden_size': (300, 1200), 'ffn_num_layers': (1, 3), 'dropout': (0.0,0.2, 0.4)}
                # parameters = {'epochs':{50,100,200},'batch_size':{16,32,64,128,256,512,1024},'depth': (2, 5), 'dropout': (0.0, 0.2, 0.4)}
                # parameters = {'batch_size':{16,32,64,128,256,512,1024},'depth': (2, 5), 'dropout': (0.0, 0.2, 0.4)}
                # parameters = {'batch_size':{16,32,64,128,256,512,1024},'depth': (2, 5), 'dropout': (0.0, 0.2, 0.4)}
                # logger.info("parameters:%s"%parameters)
            
            
                raw_args = [
                  '--data_path',data_path,
                  '--dataset_type', dataset_type,
                  '--save_dir', "hyperopt/"+data_name,
                  '--target_columns', target_column,
                  '--smiles_column',smiles_column,
                  '--seed', str(SEED),
                  '--loss_function','mse',
                 # '--checkpoint_dir','ck_dir',
                  '--split_type', 'random', # Literal['random', 'scaffold_balanced', 'predetermined', 'crossval', 'cv', 'cv-no-test', 'index_predetermined', 'random_with_repeated_smiles'] = 'random'
                  '--split_sizes', '0.8', '0.1', '0.1',
                  '--num_folds', str(NUM_FOLDS),
                  '--epochs', '%s'%epochs,
                  '--metric', 'rmse',
                  '--empty_cache',
                  '--extra_metrics', 'r2', 'mae',
                  '--batch_size','%s'%batch_size,
                  '--save_smiles_splits',
                  '--config_save_path',"config/"+data_name,
                  '--num_iter','20',
                  '--empty_cache',    
                         '--gpu','1',    
                  # '--no_features_scaling',
                  '--features_generator','morgan',
                  '--search_parameter_keywords','basic',
                  '--quiet'] 
    # Hyperopt
    # with patch('sys.argv', raw_args):
                # command_line = ' '.join(raw_args[:])
                # logger.info(f'python hyperparameter_optimization.py {command_line}')

                args = chemprop.args.HyperoptArgs().parse_args(raw_args)
                logger.info("args:%s"%args)
                hyperopt(args=args)

                # for parameter, (min_value, max_value) in parameters.items():
                #     self.assertTrue(min_value <= config[parameter] <= max_value)
    # args = chemprop.args.TrainArgs().parse_args(arguments)
    # mean_score, std_score = chemprop.train.cross_validate(args=args, train_func=chemprop.train.run_training)
    logger.info(raw_args)
    logger.info("data_name:%s data_path:%s target_column:%s"%(data_name,data_path,target_column))
    # logger.info("mean_score:%s std_score:%s"%(mean_score,std_score))

import datetime
now_time = datetime.datetime.now()
data_str = now_time.strftime('%Y_%m_%d')
dataset_dir= '/data/project/pbpk/data/'
dataset_name='caco2_train_set_6083.csv'
data_path=dataset_dir+dataset_name
logger.info(dataset_dir+dataset_name)
smiles_column='SMILES'
# target_column= 'human CL (mL/min/kg)_log10'
target_column= 'caco2_log10'
# target_column= 'Y'
data_name='caco2_log10_morgan_%s_%s'%(train_param,data_str)
logger.info("data_name:%s"%data_name)
stopwatch = StopWatch()
run_hyperopt(data_path,data_name,target_column,smiles_column,batch_size=32,epochs=100)
stopwatch.stop('hyper search finish: ')
# python hyperparameter_optimization.py /data/project/pbpk/data/fup_data_human_2668.csv --dataset_type regression --save_dir result/fup_log10 
# --target_columns fup_log10 --smiles_column SMILES --loss_function mse --split_type random --split_sizes 0.8 0.1 0.1 --num_folds 5 --epochs 20 
# --batch_size 32 --save_smiles_splits --config_save_path config/fup_log10 --num_iter 10 --empty_cache --quiet


Overwriting hyperopt/train_caco2_morgan_search.py


In [ ]:
%run hyperopt/train_caco2_search.py

In [ ]:
%run hyperopt/train_caco2_rdkit_earch.py